<a href="https://colab.research.google.com/github/junkyuhufs/stats/blob/main/WoS_text_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##텍스트파일 합치기

In [ ]:
from google.colab import files
import io

# 🔹 파일 여러 개 업로드
uploaded = files.upload()

# 🔹 파일 내용을 병합할 리스트
merged_lines = []

# 🔹 업로드된 모든 파일을 순서대로 병합
for filename in uploaded.keys():
    print(f"📄 처리 중: {filename}")
    with open(filename, "r", encoding="utf-8") as f:
        merged_lines.extend(f.readlines())
        merged_lines.append("\n")  # 파일 간 구분을 위한 빈 줄 추가 (선택 사항)

# 🔹 병합된 파일 저장
merged_filename = "merged_output.txt"
with open(merged_filename, "w", encoding="utf-8") as f:
    f.writelines(merged_lines)

print(f"✅ 병합 완료: {merged_filename}")

# 🔹 병합된 파일 다운로드
files.download(merged_filename)


##DT==Article만 남기기

In [ ]:
# 1. 파일 업로드 위젯
from google.colab import files
uploaded = files.upload()

# 2. 업로드된 파일 이름 가져오기
uploaded_filename = next(iter(uploaded))
output_filename = "1_500_article_only.txt"

# 3. DT가 "Article"인 논문만 남기고 나머지 제거
total_records = 0
removed_records = 0
filtered_lines = []
inside_record = False
keep_record = False
current_record = []

with open(uploaded_filename, "r", encoding="utf-8") as f:
    for line in f:
        stripped_line = line.strip()
        if stripped_line.startswith("PT "):
            total_records += 1
            inside_record = True
            keep_record = False
            current_record = [line]
        elif stripped_line.startswith("ER") and inside_record:
            current_record.append(line)
            inside_record = False
            if keep_record:
                filtered_lines.extend(current_record)
            else:
                removed_records += 1
        elif inside_record:
            current_record.append(line)
            if stripped_line.startswith("DT ") and stripped_line[3:].strip() == "Article":
                keep_record = True
        else:
            filtered_lines.append(line)  # 파일 맨 앞이나 끝부분에 있는 부가 정보

# 4. 결과 파일 저장
with open(output_filename, "w", encoding="utf-8") as f:
    f.writelines(filtered_lines)

# 5. 결과 출력
print(f"총 논문 수: {total_records}")
print(f"삭제된 논문 수 (DT != 'Article'): {removed_records}")
print(f"남은 논문 수: {total_records - removed_records}")
print(f"파일 저장 완료: {output_filename}")

# 6. 다운로드 링크 생성
files.download(output_filename)


##참고문헌 중 저자불분명삭제

In [ ]:
# 파일 업로드 위젯 (Colab에서 파일 업로드)
from google.colab import files
uploaded = files.upload()

# 파일 이름 (사용자가 업로드한 파일 이름 자동 추출)
import io

uploaded_filename = next(iter(uploaded))
file_path = uploaded_filename
output_path = "1_500_no_anonymous_preserve_cr.txt"

# [Anonymous] 라인 삭제 + CR 라벨 유지
removed_count = 0
cleaned_lines = []
inside_cr_section = False

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        stripped_line = line.strip()
        if stripped_line.startswith("CR"):
            inside_cr_section = True
            if "[Anonymous]" in stripped_line:
                removed_count += 1
                cleaned_lines.append("CR\n")  # CR은 유지
                continue
        elif inside_cr_section and stripped_line == "":
            inside_cr_section = False
        elif inside_cr_section:
            if "[Anonymous]" in stripped_line:
                removed_count += 1
                continue
        cleaned_lines.append(line)

# 새 파일로 저장
with open(output_path, "w", encoding="utf-8") as f:
    f.writelines(cleaned_lines)

# 결과 출력
print(f"[Anonymous] 라인이 {removed_count}개 삭제되었습니다.")
print(f"정리된 파일이 저장되었습니다: {output_path}")

# 결과저장
files.download("1_500_no_anonymous_preserve_cr.txt")
